In [1]:
# ✅ 필요한 라이브러리 설치 (처음 1회만 실행)
!pip install pandas konlpy scikit-learn wordcloud matplotlib ipywidgets --quiet


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextens

In [4]:
# ✅ 기본 모듈 import
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from ipywidgets import interact, Dropdown
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# ✅ CSV 데이터 로드
df = pd.read_csv("병합된_청원_데이터.csv")

# ✅ 날짜 처리
df["등록일"] = pd.to_datetime(df["접수일"], errors="coerce")
df["월"] = df["등록일"].dt.to_period("M")

# ✅ Okt 형태소 분석기
okt = Okt()

# ✅ 문서 생성 (전체 & 월별)
monthly_docs = {}
all_docs = []

for period, group in df.groupby("월"):
    texts = (group["청원명"].fillna("") + " " + group["청원요지"].fillna("")).tolist()
    joined_text = " ".join(texts)
    nouns = [word for word in okt.nouns(joined_text) if 2 <= len(word) <= 5]
    doc = " ".join(nouns)
    all_docs.append(doc)
    monthly_docs[str(period)] = doc

# ✅ TF-IDF 계산 (전체 문서 기준)
vectorizer = TfidfVectorizer(token_pattern=r"[가-힣]{2,5}")
tfidf_matrix = vectorizer.fit_transform(all_docs)
feature_names = vectorizer.get_feature_names_out()
global_tfidf = dict(zip(feature_names, tfidf_matrix.toarray().sum(axis=0)))

In [ ]:
# ✅ 불용어 정의 (트렌드 분석 방해 단어 제거)
stopwords = set([
    "청원", "요청", "의원", "정부", "국민", "관련", "문제", "조치", "대책", "해결", "필요", "부탁", "보장", "의혹", "언제", "통해", "일치",
    "검토", "확인", "요구", "마련", "처리", "사항", "위해", "위한", "대한", "있도록", "합니다", "대한민국", "국가", "국내", "발의", "개인", 
    "제한", "영향", "안정", "해당", "정책", "우려", "활동", "사례", "우선", "작성", "활용", "모든", "위협", "또한", "대해", "정의", "근거",
    "상황", "정성", "야기", "다수", "주요", "효과", "도입", "방법", "적용", "진행", "최근", "전면", "판단", "유지", "과정", "각종", "원래", 
    "현재", "취지", "체제", "존속", "당초", "발전"
])

# ✅ 분석 및 시각화 함수
def show_wordcloud(selected_month):
    if selected_month not in monthly_docs:
        print("❌ 해당 월 데이터가 없습니다.")
        return
    
    doc = monthly_docs[selected_month]
    words = doc.split()

    # 불용어 제거
    filtered_words = [word for word in words if word not in stopwords]

    # TF-IDF 점수 반영
    word_scores = {word: global_tfidf.get(word, 0) for word in filtered_words}

    # ✅ 키워드 순위 출력
    top_keywords = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)[:30]
    print(f"\n✅ {selected_month} 키워드 TOP 30 (불용어 제거)")
    for word, score in top_keywords:
        print(f"{word}: {score:.4f}")

    # ✅ 워드클라우드 생성 및 시각화
    wordcloud = WordCloud(
        font_path="/Library/Fonts/AppleGothic.ttf",  # Windows: "malgun.ttf"
        width=800,
        height=600,
        background_color='white'
    ).generate_from_frequencies(word_scores)

    plt.figure(figsize=(10, 8))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"{selected_month} 청원 트렌드 워드클라우드", fontsize=20)
    plt.show()

    # ✅ 이미지 저장
    wordcloud.to_file(f"청원_트렌드_{selected_month}.png")
    print(f"✅ 워드클라우드 이미지 저장 완료: 청원_트렌드_{selected_month}.png")

# ✅ 드롭다운 UI 연결
month_dropdown = Dropdown(
    options=sorted(monthly_docs.keys()),
    description='분석할 월:',
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)

interact(show_wordcloud, selected_month=month_dropdown)


interactive(children=(Dropdown(description='분석할 월:', layout=Layout(width='300px'), options=('1988-02', '1988-0…

<function __main__.show_wordcloud(selected_month)>